In [37]:
import json
with open('../hotpot_train_with_neg_v0.json','r') as f:
    train_data = [json.loads(line) for line in f]        

In [38]:
train_data[4]

{'question': 'Cadmium Chloride is slightly soluble in this chemical, it is also called what?',
 'answers': ['alcohol'],
 'type': 'bridge',
 'pos_paras': [{'title': 'Ethanol',
   'text': 'Ethanol, also called alcohol, ethyl alcohol, and drinking alcohol, is a compound and simple alcohol with the chemical formula C2H5OH . Its formula can be written also as CH3 −CH2 −OH or C2H5 −OH (an ethyl group linked to a hydroxyl group), and is often abbreviated as EtOH. Ethanol is a volatile, flammable, colorless liquid with a slight characteristic odor. It is used as a drug and is the principal type of alcohol found in alcoholic drinks.'},
  {'title': 'Cadmium chloride',
   'text': 'Cadmium chloride is a white crystalline compound of cadmium and chlorine, with the formula CdCl. It is a hygroscopic solid that is highly soluble in water and slightly soluble in alcohol. Although it is considered to be ionic, it has considerable covalent character to its bonding. The crystal structure of cadmium chlori

In [39]:
len(train_data)

90447

In [40]:
with open('hotpot_train_v1.1.json','r') as f:
    train_data_all = json.load(f)

In [41]:
train_data_all[0]

{'supporting_facts': [["Arthur's Magazine", 0], ['First for Women', 0]],
 'level': 'medium',
 'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'context': [['Radio City (Indian radio station)',
   ["Radio City is India's first private FM radio station and was started on 3 July 2001.",
    ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).',
    ' It plays Hindi, English and regional songs.',
    ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.',
    ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.',
    ' The Radio station currently plays a mix of Hindi and Regional music.',
    ' Abraham Thomas is the CEO of the company.']]

In [42]:
from tqdm import tqdm
cf_set = list()
count = 0
for entry in tqdm(train_data_all):
    question = entry['question']
    answer = [entry['answer']]
    support_ctx = {title : sent_id for title, sent_id in entry['supporting_facts']}
    pos_psgs = list()
    cf_psgs = list()
    for ctx in entry['context']:
        ctx_title = ctx[0]
        if ctx_title in support_ctx:
            pos_psg = [ctx[0], ''.join(ctx[1])]
            answer_sent = support_ctx[ctx_title]
            pos_psgs.append(pos_psg)
            cf_psg = [ctx[0],''.join(ctx[1][:answer_sent]+ctx[1][answer_sent+1:])]
            #if len(cf_psg) > 0:
            cf_psgs.append(cf_psg)

    if len(cf_psgs) <= 0:
        print(question)
        count += 1

    cf_set.append({
        'question': question,
        'answers': answer,
        'type': entry['type'],
        'pos_paras': pos_psgs,
        'cf_paras': cf_psgs,
    })
print(count)

100%|██████████| 90447/90447 [00:01<00:00, 45774.27it/s]

0


In [43]:
cf_set[3]['cf_paras']

[['Ewan MacColl', ''],
 ['Peggy Seeger',
  'Margaret "Peggy" Seeger (born June 17, 1935) is an American folksinger.']]

In [44]:
for i, entry in enumerate(cf_set):
    cf_par_1, cf_par_2 = entry['cf_paras']
    if not cf_par_1[0] and not cf_par_1[1]:
        print(i)
    if not cf_par_2[0] and not cf_par_2[1]:
        print(i)

In [45]:
cf_dict = {entry['question']:entry['cf_paras'] for entry in cf_set}
for it in train_data:
    cf_passages = cf_dict[it['question']]
    cf_psg_dict = [
        {
            'title': psg[0],
            'text': psg[1]
        } for psg in cf_passages
    ]
    it['cf_paras'] = cf_psg_dict

In [46]:
train_data[0]

{'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answers': ["Arthur's Magazine"],
 'type': 'comparison',
 'pos_paras': [{'title': "Arthur's Magazine",
   'text': 'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book".'},
  {'title': 'First for Women',
   'text': "First for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies."}],
 'neg_paras': [{'title': 'Knowledge (magazine)',
   'text': "Knowledge: An Illustrated Magazine of Science was a British popular science magazine published from 1881 to 1918. Founded by astronomer Richard A. Proctor, the magazine star

In [47]:
clean_entries = list()
for entry in train_data:
    if len(entry['neg_paras']) > 1:
        clean_entries.append(entry)

In [48]:
with open('hotpot_train_counterfactual.json','w') as f:
    for li in clean_entries:
        f.write(json.dumps(li) + '\n')


In [ ]:
import math
import re
import random
import nltk
random.seed(0)

make_random_mask = False
exact_answers = []
not_include_answer = []
span_len = 16
WINDOWING=False

def find_random_idx(ans_start_tok, answer_end_tok,txt):

    max_bound = max(len(re.split(' ',txt))-(answer_end_tok-ans_start_tok), answer_end_tok)
    middle_bound = max(ans_start_tok-(answer_end_tok-ans_start_tok),0)

    rand_candidates = list(range(middle_bound))+list(range(answer_end_tok,max_bound))

    rand_start_idx = random.choice(rand_candidates)
    rand_end_idx = rand_start_idx + (answer_end_tok-ans_start_tok)
    return rand_start_idx, rand_end_idx

masked_spans=[]

def find_and_mask(answers, txt):
    masked_txt = txt
    masked=False
    for ans in answers:
        ans_len_char = len(ans)
        ans = re.sub(r'([^a-zA-Z0-9,\.!\? -])',r'\\\1',ans)

        ans_split = ans.split(' ')
        ans_len_token = len(ans_split)

        re_ans = '( )?'.join(ans_split)
        re_ans = re.compile(re_ans,re.I)
        start_idxs = list(re.finditer(re_ans, masked_txt))
        ans_spans = [(-1,0)] + [(start_idx.start(), start_idx.start() + ans_len_char) for start_idx in start_idxs] + [(len(masked_txt),-1)]

        if len(start_idxs) > 0:
            new_masked_txt = ''
            for i, _ in enumerate(ans_spans[:-1]):
                s_idx = ans_spans[i][1]
                e_idx = ans_spans[i+1][0]
                new_masked_txt += masked_txt[s_idx:e_idx]+' '
            masked_txt = new_masked_txt
            masked=True

    if masked:
        return masked_txt
    return None

In [ ]:
for entry in tqdm(cf_set):
    new_cf_paras = list()
    for cf_ctx in entry['cf_paras']:
        if find_and_mask(entry['answers'], cf_ctx[0]):
            new_title = find_and_mask(entry['answers'], cf_ctx[0])
            new_cf_ctx = [new_title, cf_ctx[1]]
            new_cf_paras.append(new_cf_ctx)
        else:
            new_cf_paras.append(cf_ctx)
    entry['cf_paras'] = new_cf_paras

### Dev File

In [6]:
import json

train_data = list()
with open('../hotpot_dev_with_neg_v0.json','r') as f:
    for line in f:
        train_data.append(json.dumps)

In [36]:
for entry in dev_data:
    print(entry.keys())
    if len(entry['neg_paras']) < 10:
        print(entry)
    break

dict_keys(['question', 'answers', 'type', 'pos_paras', 'neg_paras', '_id'])


### Corpus file

In [1]:

import json
with open('../hotpot_index/wiki_id2doc.json','r') as f:
    corpus_data = json.load(f)

In [2]:
len(corpus_data.keys())

5233329

### Convert to DPR format

In [5]:
import json

with open('../curatedtrec-dev.json','r') as f:
    trec_dev = json.load(f)

In [6]:
trec_dev[0]

{'dataset': 'curatedtrec',
 'question': 'What site did Lindbergh begin his flight from in 1927?',
 'answers': ['Long Island|New\\s?York|Roosevelt Field'],
 'positive_ctxs': [{'title': 'Charles Lindbergh',
   'text': 'Charles Lindbergh Charles Augustus Lindbergh (February 4, 1902 – August 26, 1974) was an American aviator, military officer, author, inventor, explorer, and environmental activist. At age 25 in 1927, he went from obscurity as a [[U.S. Air Mail]] pilot to instantaneous world fame by winning the [[Orteig Prize]]: making a nonstop flight from [[Roosevelt Field (airport)|Roosevelt Field]], [[Long Island]], [[New York (state)|New York]], to [[Paris]], France. Lindbergh covered the -hour, flight alone in a single-engine purpose-built [[Ryan Airline Company|Ryan]] [[monoplane]], the "[[Spirit of St. Louis]]". This was not the [[Transatlantic flight of Alcock and Brown|first flight between North America and',
   'score': 20.811531,
   'title_score': 0,
   'psg_id': '871627'},
  {'

In [7]:
import json

train_data = list()
with open('../hotpot_train_counterfactual.json','r') as f:
    for line in f:
        train_data.append(json.loads(line.strip()))
        

In [8]:
train_data[0]

{'question': "Which magazine was started first Arthur's Magazine or First for Women?",
 'answers': ["Arthur's Magazine"],
 'type': 'comparison',
 'pos_paras': [{'title': "Arthur's Magazine",
   'text': 'Arthur\'s Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century. Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others. In May 1846 it was merged into "Godey\'s Lady\'s Book".'},
  {'title': 'First for Women',
   'text': "First for Women is a woman's magazine published by Bauer Media Group in the USA. The magazine was started in 1989. It is based in Englewood Cliffs, New Jersey. In 2011 the circulation of the magazine was 1,310,696 copies."}],
 'neg_paras': [{'title': 'Knowledge (magazine)',
   'text': "Knowledge: An Illustrated Magazine of Science was a British popular science magazine published from 1881 to 1918. Founded by astronomer Richard A. Proctor, the magazine star

In [9]:
train_data[0]['question']

"Which magazine was started first Arthur's Magazine or First for Women?"

In [10]:
with open('../wiki_id2doc.json','r') as f:
    corpus = json.load(f)

In [11]:
from tqdm import tqdm

doc2id = dict()
for k,v in tqdm(corpus.items()):
    doc2id[v['title'].strip()] = k

  0%|          | 0/5233329 [00:00<?, ?it/s]

100%|██████████| 5233329/5233329 [00:04<00:00, 1096661.52it/s]


In [12]:
import random

dpr_train_data = list()
for entry in tqdm(train_data):
    dpr_entry = dict()
    dpr_entry['dataset'] = 'hotpotqa'
    dpr_entry['question'] = entry['question']
    dpr_entry['answers'] = entry['answers']
    
    if len(entry['pos_paras']) < 2:
        print(entry['pos_paras'])
    if len(entry['cf_paras']) < 2:
        print(entry['cf_paras'])

    dpr_entry['positive_ctxs'] = [{
        'title' : para['title'],
        'text' : para['text'],
        'score' : 100.0,
        'title_score' : 100.0,
        'psg_id' : 0,
    } for para in entry['pos_paras']]

    dpr_entry['hard_negative_ctxs'] = [{
        'title' : para['title'],
        'text' : para['text'],
        'score' : 100.0,
        'title_score' : 100.0,
        'psg_id' : 0,
    } for para in entry['neg_paras']]

    dpr_entry['cf_negative_ctxs'] = [{
        'title' : para['title'],
        'text' : para['text'],
        'score' : 100.0,
        'title_score' : 100.0,
        'psg_id' : 0,
    } for para in entry['cf_paras']]

    dpr_train_data.append(dpr_entry)

100%|██████████| 90445/90445 [00:01<00:00, 53894.60it/s]


In [ ]:
for entry in dpr_train_data:
    if len(entry['cf_negative_ctxs']) < 2:
        print(entry)

In [13]:
with open('../hotpot_train_counterfactual_dpr.json','w') as f:
    json.dump(dpr_train_data,f)

In [37]:
with open('../hotpot_dev_with_neg_v0.json','r') as f:
    dev_data = [json.loads(line.strip()) for line in f]

In [38]:
dev_data[0]

{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'answers': ['yes'],
 'type': 'comparison',
 'pos_paras': [{'title': 'Ed Wood',
   'text': 'Edward Davis Wood Jr. (October 10, 1924\xa0– December 10, 1978) was an American filmmaker, actor, writer, producer, and director.'},
  {'title': 'Scott Derrickson',
   'text': 'Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. He lives in Los Angeles, California. He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."'}],
 'neg_paras': [{'title': 'That Darn Cat (1997 film)',
   'text': 'That Darn Cat is a 1997 American mystery comedy film starring Christina Ricci and Doug E. Doug. It is a remake of the 1965 film "That Darn Cat!", which in turn was based on the book "Undercover Cat" by Gordon and Mildred Gordon. It is directed by British TV ve

In [40]:
dpr_dev_data = list()
for entry in tqdm(dev_data):
    dpr_dev_entry = {
        'dataset' : 'hotpotqa',
        'question' : entry['question'],
        'answers' : entry['answers']
    }

    dpr_dev_entry['positive_ctxs'] = [{
        'title' : para['title'],
        'text' : para['text'],
        'score' : 100.0,
        'title_score' : 100.0,
        'psg_id' : 0,
    } for para in entry['pos_paras']]

    dpr_dev_entry['hard_negative_ctxs'] = [{
        'title' : para['title'],
        'text' : para['text'],
        'score' : 100.0,
        'title_score' : 100.0,
        'psg_id' : 0,
    } for para in entry['neg_paras']]

    dpr_dev_data.append(dpr_dev_entry)

100%|██████████| 7405/7405 [00:00<00:00, 53013.43it/s]


In [41]:
with open('../hotpot_dev_counterfactual_dpr.json','w') as f:
    json.dump(dpr_dev_data,f)

In [1]:
import json
with open('../hotpot_train_counterfactual.json','r') as f:
    orig_train_data = [json.loads(line) for line in f]

In [4]:
for entry in orig_train_data:
    if len(entry['pos_paras']) < 2:
        print(entry['pos_paras'])

### Build DPR Corpus and Test set

In [1]:
import json
with open('../wiki_id2doc.json','r') as f:
    corpus_file = json.load(f)

In [2]:
from tqdm import tqdm

dpr_corpus = list()
for pid, psg in tqdm(corpus_file.items()):
    dpr_corpus.append([pid, psg['text'], psg['title']])

  0%|          | 0/5233329 [00:00<?, ?it/s]

100%|██████████| 5233329/5233329 [00:08<00:00, 606391.16it/s] 


In [3]:
import csv
with open('psgs_hotpot.csv','w') as f:
    writer = csv.writer(f, delimiter='\t')
    for line in tqdm(dpr_corpus):
        writer.writerow(line)

100%|██████████| 5233329/5233329 [00:41<00:00, 124738.00it/s]


In [5]:
import json
with open('../hotpot_qas_val.json','r') as f:
    eval_data = [json.loads(line) for line in f]

In [7]:
eval_data[0]

{'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 '_id': '5a8b57f25542995d1e6f1371',
 'answer': ['yes'],
 'sp': ['Scott Derrickson', 'Ed Wood'],
 'type': 'comparison'}

In [8]:
len(eval_data)

7405

### Retriever Results to Reader Input

In [1]:
import json
with open('../dev_retrieval_top100_sp.json','r') as f:
    reader_input_format = [json.loads(line) for line in f]

In [2]:
reader_input_format[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'type': 'comparison',
 'sp': [{'title': 'Scott Derrickson',
   'sents': ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
    ' He lives in Los Angeles, California.',
    ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."'],
   'sp_sent_ids': [0]},
  {'title': 'Ed Wood',
   'sents': ['Edward Davis Wood Jr. (October 10, 1924\xa0– December 10, 1978) was an American filmmaker, actor, writer, producer, and director.'],
   'sp_sent_ids': [0]}],
 'answer': ['yes'],
 'candidate_chains': [[{'title': 'Scott Derrickson',
    'sents': ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
     ' He lives in Los Angeles, California.',
     ' He is best known 

In [12]:
with open('../checkpoint_hotpotqa_aadpr_reproduce_vanilla_epoch39_on_dpr_hotpotqa_hotpot-eval/scores.json','r') as f:
    aadpr_scores = json.load(f)

In [13]:
import nltk
import copy
from tqdm import tqdm

aadpr_scores_for_mdr = list()
assert len(reader_input_format) == len(aadpr_scores)
for inp, scr in tqdm(zip(reader_input_format, aadpr_scores)):
    #assert inp['question'] == scr['question'], (inp['question'], scr['question'])
    cand_scrs = [{
        'title': ctx['title'],
        'sents' : nltk.sent_tokenize(ctx['text'])
    } for ctx in scr['ctxs']]
    new_entry = copy.deepcopy(inp)
    new_entry['candidate_chains'] = [cand_scrs]
    aadpr_scores_for_mdr.append(new_entry)

7405it [00:57, 128.15it/s]


In [14]:
with open('aadpr_vanilla_reader_input.json','w') as f:
    for line in aadpr_scores_for_mdr:
        f.write(json.dumps(line) + '\n')

MDR Retrieval Results to Reader Input

In [94]:
import json
with open('../hotpot_dpr_ours/scores.json','r') as f:
    val_data = [json.loads(line) for line in f]

In [95]:
val_data[0]

{'_id': '5a8b57f25542995d1e6f1371',
 'question': 'Were Scott Derrickson and Ed Wood of the same nationality?',
 'candidate_chains': [[{'title': 'Scott Derrickson',
    'text': 'Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer. He lives in Los Angeles, California. He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."',
    'sents': ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.',
     ' He lives in Los Angeles, California.',
     ' He is best known for directing horror films such as "Sinister", "The Exorcism of Emily Rose", and "Deliver Us From Evil", as well as the 2016 Marvel Cinematic Universe installment, "Doctor Strange."']},
   {'title': 'Ed Wood',
    'text': 'Edward Davis Wood Jr. (October 10, 1924\xa0– December 10, 1978) was an American filmmaker, acto

In [96]:
import json
with open('../dev_retrieval_top100_sp.json','r') as f:
    reader_input_format = [json.loads(line) for line in f]

In [97]:
import nltk
import copy
from tqdm import tqdm

assert len(val_data) == len(reader_input_format)
aadpr_mdr_scores_for_reader = list()
for inp, scr in tqdm(zip(reader_input_format, val_data)):
    assert inp['question'] == scr['question']
    new_entry = copy.deepcopy(inp)
    new_entry['candidate_chains'] = scr['candidate_chains']
    aadpr_mdr_scores_for_reader.append(new_entry)

0it [00:00, ?it/s]

7405it [00:06, 1146.69it/s]


In [98]:
with open('dpr_mdr_reader_input.json','w') as f:
    for line in aadpr_mdr_scores_for_reader:
        f.write(json.dumps(line) + '\n')

In [89]:
idx = 12
inp = reader_input_format[idx]
val = val_data[idx]
inp['question'], val['question'], inp['answer']

('What year did Guns N Roses perform a promo for a movie starring Arnold Schwarzenegger as a former New York Police detective?',
 'What year did Guns N Roses perform a promo for a movie starring Arnold Schwarzenegger as a former New York Police detective?',
 ['1999'])

In [90]:
cand_idx = 0
inp['candidate_chains'][cand_idx]

[{'title': 'Nice Guy Eddie',
  'sents': ['"For the Tarantino film character, see Reservoir Dogs"']},
 {'title': 'Reservoir Dogs',
  'sents': ['Reservoir Dogs is a 1992 American crime thriller film and the feature-length debut of writer and director Quentin Tarantino.',
   ' It features Harvey Keitel, Michael Madsen, Steve Buscemi, Chris Penn, Lawrence Tierney, Tim Roth, Tarantino, and criminal-turned-author Edward Bunker as members of a botched diamond heist.',
   ' The film depicts the events before and after the heist.',
   ' Kirk Baltz, Randy Brooks and Steven Wright also play supporting roles.',
   " It incorporates many themes that have become Tarantino's hallmarks: violent crime, pop culture references, profanity, and nonlinear storytelling."]}]

In [91]:
val['candidate_chains'][cand_idx]

[{'title': 'The Spy in the Green Hat',
  'text': 'The Spy in the Green Hat is a 1967 feature-length film version of "The Man from U.N.C.L.E."\' s third season two-part episode "The Concrete Overcoat Affair". The episodes were originally broadcast in the United States on November 25, 1966 and December 2, 1966 on NBC. The film was directed by Joseph Sargent and written by Peter Allan Fields with the story by David Victor. It, as does the television series, stars Robert Vaughn and David McCallum. It is the fifth such feature film that used as its basis a reedited version of one or more episodes from the series.',
  'sents': ['The Spy in the Green Hat is a 1967 feature-length film version of "The Man from U.N.C.L.E."\' s third season two-part episode "The Concrete Overcoat Affair".',
   ' The episodes were originally broadcast in the United States on November 25, 1966 and December 2, 1966 on NBC.',
   ' The film was directed by Joseph Sargent and written by Peter Allan Fields with the stor

In [93]:
len(val['candidate_chains'])

20